In [31]:
#!pip install statsmodels

In [32]:
import os
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
import plotly.express as px

In [33]:
data_dir = '../data'
predictions_path = os.path.join(data_dir, 'predictions.tsv')
gos_path = os.path.join(data_dir, 'gos.tsv')
go_ontology_path = os.path.join(data_dir, 'go_ontology.tsv')

In [34]:
predictions = pd.read_csv(predictions_path, sep='\t')
predictions.head()

,taxid1,taxid1_label,source_color,source_shape,source,source_name,taxid2,taxid2_label,target_color,target_shape,target,target_name,experimental_evidence_score,databases_evidence_score,weight,group1,group2,edge_type
0,5691,Trypanosoma brucei,#bc80bd,diamond,5691.EAN79407,EAN79407,9606,Homo sapiens,#525252,dot,9606.ENSP00000339740,CAMK2D,0.000,0.77,0.385,KOG0039,KOG0033,inter-species
1,5691,Trypanosoma brucei,#bc80bd,diamond,5691.EAN79407,EAN79407,9606,Homo sapiens,#525252,dot,9606.ENSP00000362057,NOX1,0.000,0.77,0.385,KOG0039,KOG0033,inter-species
2,5691,Trypanosoma brucei,#bc80bd,diamond,5691.EAN79407,EAN79407,9606,Homo sapiens,#525252,dot,9606.ENSP00000475084,DUOX2,0.000,0.77,0.385,KOG0039,KOG0033,inter-species
3,5671,Leishmania infantum,#e31a1c,diamond,5671.XP_001467017.1,XP_001467017.1,9606,Homo sapiens,#525252,dot,9606.ENSP00000339740,CAMK2D,0.994,0.00,0.497,KOG0078,KOG0033,inter-species
4,5671,Leishmania infantum,#e31a1c,diamond,5671.XP_001467017.1,XP_001467017.1,9606,Homo sapiens,#525252,dot,9606.ENSP00000300935,RAB8A,0.994,0.00,0.497,KOG0078,KOG0033,inter-species


In [94]:
predictions[(predictions['taxid1_label'] == 'Plasmodium falciparum') & (predictions['weight']>0.4)].drop_duplicates(['source', 'target']).shape

(250, 18)

In [95]:
tissues = pd.read_csv('../data/tissues_cell_types.tsv', sep='\t', header=0)
pred_tissues = pd.merge(predictions, tissues.rename({'Gene': 'target'}, axis=1), on='target', how='left')

In [96]:
pred_tissues[(pred_tissues['taxid1_label'] == 'Plasmodium falciparum') & (pred_tissues['weight']>0.4)].drop_duplicates(['source', 'target']).shape

(250, 24)

In [35]:
gos = pd.read_csv(gos_path, sep='\t')
gos.head()

,#string_protein_id,description,taxid
0,9606.ENSP00000000233,Transport,9606
1,9606.ENSP00000000233,Intracellular protein transport,9606
2,9606.ENSP00000000233,"Retrograde vesicle-mediated transport, golgi t...",9606
3,9606.ENSP00000000233,Protein localization,9606
4,9606.ENSP00000000233,Cellular process,9606


In [36]:
net = predictions[(predictions['taxid1'].isin([5691, 9606])) & (predictions['weight'] >=0.4)]
net.head()

,taxid1,taxid1_label,source_color,source_shape,source,source_name,taxid2,taxid2_label,target_color,target_shape,target,target_name,experimental_evidence_score,databases_evidence_score,weight,group1,group2,edge_type
74,5691,Trypanosoma brucei,#bc80bd,diamond,5691.EAN79407,EAN79407,9606,Homo sapiens,#525252,dot,9606.ENSP00000263025,MAPK3,0.354,0.866,0.6100,KOG0039,KOG0660,inter-species
75,5691,Trypanosoma brucei,#bc80bd,diamond,5691.EAN79407,EAN79407,9606,Homo sapiens,#525252,dot,9606.ENSP00000215832,MAPK1,0.354,0.866,0.6100,KOG0039,KOG0660,inter-species
109,5691,Trypanosoma brucei,#bc80bd,diamond,5691.EAN79407,EAN79407,9606,Homo sapiens,#525252,dot,9606.ENSP00000315768,STAT2,0.259,0.866,0.5625,KOG0039,KOG3667,inter-species
110,5691,Trypanosoma brucei,#bc80bd,diamond,5691.EAN79407,EAN79407,9606,Homo sapiens,#525252,dot,9606.ENSP00000264657,STAT3,0.259,0.866,0.5625,KOG0039,KOG3667,inter-species
121,5691,Trypanosoma brucei,#bc80bd,diamond,5691.EAN79407,EAN79407,9606,Homo sapiens,#525252,dot,9606.ENSP00000360293,SORBS1,0.467,0.866,0.6665,KOG0039,KOG4225,inter-species


In [37]:
net_gos = gos[gos['taxid'].isin([5691, 9606])].groupby('description').filter(lambda x: len(x)< 500)
net_gos.head()

,#string_protein_id,description,taxid
2,9606.ENSP00000000233,"Retrograde vesicle-mediated transport, golgi t...",9606
13,9606.ENSP00000000233,Golgi vesicle transport,9606
21,9606.ENSP00000000412,Peptide secretion,9606
22,9606.ENSP00000000412,Protein targeting,9606
23,9606.ENSP00000000412,Protein targeting to lysosome,9606


In [38]:
nodes = net['source'].unique().tolist() + net['target'].unique().tolist()

In [39]:
total_nodes = len(nodes)

In [40]:
#C'
total_nodes

274

In [41]:
selected_gos = net_gos[net_gos['#string_protein_id'].isin(nodes)].groupby('description').filter(lambda x: len(x)> 10)['description'].unique().tolist()

In [42]:
len(selected_gos)

287

In [43]:
total_prots = len(net_gos['#string_protein_id'].unique().tolist())

In [44]:
#G
total_prots

15475

In [45]:
enrichment = []
for term in selected_gos:   
    members = net_gos[(net_gos['description'] == term)]['#string_protein_id']
    #E
    total_members = len(members)
    net_members = net_gos[(net_gos['description'] == term) & (net_gos['#string_protein_id'].isin(nodes))]['#string_protein_id']
    #A
    total_net_members = len(net_members)
    
    odd_ratio, p_value = stats.fisher_exact([[total_net_members, total_nodes - total_net_members],
                                             [total_members - total_net_members, total_prots - total_members - total_nodes - total_net_members]])
    enrichment.append([term, total_net_members, total_nodes - total_net_members, total_members - total_net_members,  total_prots - total_members - total_nodes - total_net_members, p_value, odd_ratio, ','.join(net_members)])

In [46]:
enrichment = pd.DataFrame(enrichment, columns=['go_term', 'A', 'B', 'C', 'D', 'p_value', 'odds', 'nodes'])
enrichment['fdr_bh'] = multipletests(enrichment['p_value'].tolist(), alpha=0.01, method='fdr_bh')[1]
enrichment.head()

,go_term,A,B,C,D,p_value,odds,nodes,fdr_bh
0,Eye development,15,259,350,14821,0.003615,2.452454,"9606.ENSP00000005226,9606.ENSP00000232461,9606...",0.004472
1,Regulation of cellular component size,19,255,364,14799,0.000056,3.029315,"9606.ENSP00000005226,9606.ENSP00000229264,9606...",0.000109
2,Camera-type eye development,12,262,306,14871,0.014674,2.225864,"9606.ENSP00000005226,9606.ENSP00000232461,9606...",0.016712
3,Embryonic organ development,13,261,435,14740,0.069611,1.687761,"9606.ENSP00000005226,9606.ENSP00000215832,9606...",0.074269
4,Sensory system development,15,259,356,14815,0.003857,2.410145,"9606.ENSP00000005226,9606.ENSP00000232461,9606...",0.004710


In [47]:
enrichment.shape

(287, 9)

In [67]:
enrichment = enrichment[enrichment['fdr_bh']<0.01]

In [68]:
px.scatter(enrichment, x='fdr_bh', y='odds', size='odds', color='go_term')

In [69]:
ontology = pd.read_csv(go_ontology_path, sep='\t')
ontology.head()

,parent,child,value
0,Organelle inheritance,Mitochondrion inheritance,3
1,Mitochondrion distribution,Mitochondrion inheritance,9
2,Mitochondrion organization,Mitochondrial genome maintenance,6
3,Biological_process,Reproduction,2
4,Zinc ion transmembrane transporter activity,High-affinity zinc transmembrane transporter a...,1


In [78]:
aux = ontology[(ontology['parent'].isin(enrichment['go_term'])) & (ontology['child'].isin(enrichment['go_term']))]

In [79]:
aux = pd.merge(aux.rename({'child':'go_term'}, axis=1), enrichment[['go_term', 'odds', 'fdr_bh']], on='go_term')

In [80]:
aux.head()

,parent,go_term,value,odds,fdr_bh
0,Response to oxidative stress,Response to reactive oxygen species,6,5.476750,4.629155e-07
1,Negative regulation of phosphorylation,Negative regulation of protein phosphorylation,9,2.319537,4.242182e-03
2,Neutrophil activation,Neutrophil activation involved in immune response,9,3.633616,2.839379e-07
3,Immune response-regulating cell surface recept...,Immune response-activating cell surface recept...,2,3.126843,2.491906e-04
4,Antigen processing and presentation of exogeno...,Antigen processing and presentation of exogeno...,8,3.853041,4.556691e-04


In [81]:
fig = px.treemap(aux, path=['parent', 'go_term'], values='odds', width=2000, height=1200, hover_data=['fdr_bh', 'odds'])

In [82]:
fig